In [3]:
import azureml
import json
from azureml.core.model import Model
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.68


In [4]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#   ws.write_config()
#   print('Workspace configuration succeeded. You are all set!')
#except:
#   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

Wrote the config file config.json to: c:\projects\PyTorch-YOLOv3\aml_config\config.json
Workspace configuration succeeded. You are all set!


# Managing Models
This section is devoted to managin the YOLOv3 model in Azure Machine Learning service

In [3]:
models = Model.list(workspace=ws)
for m in models:
    print(m.name, m.version)

sklearn_regression_model.pkl 1
best_ridge_model 1
sklearn_mnist 3
best_model 1
AutoMLef440bcfebest 1
YOLOv3 2
sklearn_mnist 2
sklearn_mnist 1
YOLOv3 1


In [4]:
model_file = './model'
m = Model.register(ws, model_name='YOLOv3', model_path=model_file)
m

Registering model YOLOv3


# Managing Images
Creating environment file

In [95]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
with open('./requirements.txt', 'r') as r:
    deps = r.read().split('\n')
    deps = [x.rstrip().lstrip() for x in deps]
    for item in deps:
        if len(item) > 0:
            print('Adding "pip install {}"'.format(item))
            myenv.add_pip_package(item)

yml = 'yolov3.yml'
with open(yml,'w') as f:
    print('Writing out {}'.format(yml))
    f.write(myenv.serialize_to_string())
    print('Done!')

Adding "pip install scikit-image"
Adding "pip install numpy"
Adding "pip install torch>=0.4.0"
Adding "pip install torchvision"
Adding "pip install pillow"
Adding "pip install matplotlib"
Writing out yolov3.yml
Done!


Creating Image

In [96]:
from azureml.core.image import ContainerImage, Image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                    runtime="python", 
                                    conda_file="yolov3.yml",
                                    dependencies=['./models.py', './utils'], )


image = Image.create(ws, 'yolov3', [m], image_config)

Creating image


In [98]:
imgs =  ws.images()['yolov3']
print(imgs.image_build_log_uri)

https://icebatcheastus.blob.core.windows.net/logs/hal5192063154_efe84fa0f11a4eeebd6543b698cde171.txt?sv=2017-04-17&sr=b&sig=qoXEmYVjd6/bF2dKh9%2BC4ESwxzWkW1XfZO1k2Zcs6%2Bk%3D&sp=r&se=2018-12-02T18%3A32%3A50Z


# Deploying Image
Deploy created image to ACI

In [99]:
imgs =  ws.images()['yolov3']

In [100]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "COCO",  "method" : "YOLOv3"}, 
                                               description='object detection')



In [102]:
service = Webservice.deploy_from_image(workspace=ws, 
                                       image=imgs, 
                                       name='yolov3-svc', 
                                       deployment_config=aciconfig)

Creating service


In [31]:
print(service.scoring_uri)

None


In [73]:
svc = ws.webservices()['yolov3-svc']

In [103]:
logs = svc.get_logs()
print(logs)


2018-11-02 18:42:30,359 CRIT Supervisor running as root (no user in config file)
2018-11-02 18:42:30,361 INFO supervisord started with pid 1
2018-11-02 18:42:31,362 INFO spawned: 'rsyslog' with pid 9
2018-11-02 18:42:31,363 INFO spawned: 'program_exit' with pid 10
2018-11-02 18:42:31,364 INFO spawned: 'nginx' with pid 11
2018-11-02 18:42:31,365 INFO spawned: 'iot' with pid 12
2018-11-02 18:42:31,367 INFO spawned: 'gunicorn' with pid 13
2018-11-02 18:42:31,410 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2018-11-02 18:42:32,409 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-11-02 18:42:32,409 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-11-02 18:42:32,410 INFO exited: iot (exit status 1; expected)
2018-11-02 18:42:36,563 INFO success: nginx e